In [2]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from datetime import datetime
from pathlib import Path
from matplotlib.patches import Rectangle

from utils import *

BASE_DIR = Path(os.path.abspath(''))
os.chdir(BASE_DIR)
print(f'Working directory: {os.getcwd()}')

Working directory: c:\Users\jespe\Documents\GitHub_local\ARFusion


## Functionality to perform isolate-level analysis

In [8]:
results_dict = pd.read_pickle(os.path.join(BASE_DIR, 'results', 'MM', '15_ab', 'FT_noPT_0.75_0.7', 'CV_results.pkl'))

In [9]:
results_dict.keys()

dict_keys(['train_losses', 'losses', 'accs', 'iso_accs', 'sensitivities', 'specificities', 'F1_scores', 'iso_stats', 'ab_stats'])

In [8]:
results_dict['iso_stats'][1]

,year,country,genotypes,phenotypes,num_ab,num_genotypes,num_point_mutations,ab_classes,num_masked_ab,num_masked_genes,...,num_correct,correct_S,correct_R,sensitivity,specificity,accuracy,all_correct,masked_genes,masked_ab,correct_ab
0,2019,USA,"[pmrB_Y358N=POINT, glpT_E448K=POINT]","[AMP_S, CRO_S, CIP_S, GEN_S, MEM_S, NAL_S]",6.0,2,2,"[4, 2, 3, 0, 1, 3]",4,2,...,4,4,0,NaN,1.000000,1.000000,True,"[pmrB_Y358N=POINT, glpT_E448K=POINT]","[AMP_S, CRO_S, GEN_S, MEM_S]","[True, True, True, True]"
1,2021,Netherlands,"[blaCTX-M-1, aph(3'')-Ib, mph(A), blaTEM-1, te...","[AMK_S, AMP_R, FEP_R, CTX_R, CAZ_R, CIP_S, ETP...",11.0,7,1,"[0, 4, 2, 2, 2, 3, 1, 0, 1, 1, 3]",10,4,...,10,7,3,1.0,1.000000,1.000000,True,"[aph(3'')-Ib, mph(A), glpT_E448K=POINT, aph(6)...","[AMK_S, FEP_R, CTX_R, CAZ_R, CIP_S, ETP_S, GEN...","[True, True, True, True, True, True, True, Tru..."
2,2017,Canada,"[aadA1, sat2, tet(B), lnu(F), glpT_E448K=POINT]","[AMP_S, FEP_S, CTX_S, CAZ_S, CRO_S, CIP_S, GEN...",10.0,5,1,"[4, 2, 2, 2, 2, 3, 0, 1, 1, 3]",9,3,...,8,8,0,NaN,0.888889,0.888889,False,"[aadA1, sat2, tet(B)]","[AMP_S, FEP_S, CTX_S, CAZ_S, CRO_S, CIP_S, IPM...","[False, True, True, True, True, True, True, Tr..."
3,2018,USA,[glpT_E448K=POINT],"[AMP_S, CRO_S, CIP_S, GEN_S, MEM_S, NAL_S]",6.0,1,1,"[4, 2, 3, 0, 1, 3]",5,1,...,5,5,0,NaN,1.000000,1.000000,True,[glpT_E448K=POINT],"[AMP_S, CRO_S, CIP_S, MEM_S, NAL_S]","[True, True, True, True, True]"
4,2019,USA,"[glpT_E448K=POINT, marR_S3N=POINT]","[AMK_S, AMP_S, FEP_S, CAZ_S, CRO_S, CIP_S, ETP...",12.0,2,2,"[0, 4, 2, 2, 2, 3, 1, 0, 1, 3, 1, 0]",10,1,...,10,10,0,NaN,1.000000,1.000000,True,[glpT_E448K=POINT],"[AMK_S, AMP_S, FEP_S, CAZ_S, CRO_S, ETP_S, GEN...","[True, True, True, True, True, True, True, Tru..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2049,2022,USA,"[catA1, blaCTX-M-15, qepA4, erm(B), tet(B), mp...","[AMK_S, AMP_R, FEP_S, CRO_R, CIP_R, ETP_S, GEN...",10.0,15,6,"[0, 4, 2, 2, 3, 1, 0, 3, 1, 0]",9,12,...,9,6,3,1.0,1.000000,1.000000,True,"[catA1, blaCTX-M-15, qepA4, erm(B), tet(B), mp...","[AMK_S, FEP_S, CRO_R, CIP_R, ETP_S, GEN_S, LVX...","[True, True, True, True, True, True, True, Tru..."
2050,2020,USA,"[pmrB_E123D=POINT, parE_D475E=POINT, glpT_E448...","[AMK_S, AMP_S, FEP_S, CAZ_S, CRO_S, CIP_S, ETP...",12.0,4,4,"[0, 4, 2, 2, 2, 3, 1, 0, 1, 3, 1, 0]",11,3,...,11,11,0,NaN,1.000000,1.000000,True,"[pmrB_E123D=POINT, parE_D475E=POINT, glpT_E448...","[AMK_S, FEP_S, CAZ_S, CRO_S, CIP_S, ETP_S, GEN...","[True, True, True, True, True, True, True, Tru..."
2051,2020,USA,"[pmrB_E123D=POINT, glpT_E448K=POINT, marR_S3N=...","[AMK_S, AMP_S, FEP_S, CAZ_S, CRO_S, CIP_S, ETP...",12.0,3,3,"[0, 4, 2, 2, 2, 3, 1, 0, 1, 3, 1, 0]",9,2,...,9,9,0,NaN,1.000000,1.000000,True,"[glpT_E448K=POINT, marR_S3N=POINT]","[AMP_S, FEP_S, CAZ_S, CRO_S, CIP_S, ETP_S, IPM...","[True, True, True, True, True, True, True, Tru..."
2052,2020,USA,"[pmrB_E123D=POINT, tet(A), aph(3'')-Ib, marR_S...","[AMK_S, AMP_R, FEP_S, CAZ_S, CRO_S, CIP_S, ETP...",12.0,7,3,"[0, 4, 2, 2, 2, 3, 1, 0, 1, 3, 1, 0]",10,7,...,9,9,0,0.0,1.000000,0.900000,False,"[pmrB_E123D=POINT, tet(A), aph(3'')-Ib, marR_S...","[AMK_S, AMP_R, FEP_S, CAZ_S, CRO_S, ETP_S, GEN...","[True, False, True, True, True, True, True, Tr..."
